In [1]:
from itertools import product
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.utils.data as utils
from matplotlib import image as img
from matplotlib import pyplot as plt
import os
import pandas as pd

In [2]:
def train(net, X, y, batch_size, lr, p=15, rep=5):
    X = torch.Tensor(X).float()
    N = len(X)
    y = torch.Tensor(y).long()
    train_size = int(0.9 * N)
    valid_size = N - train_size
    train_index, valid_index = torch.utils.data.random_split(
        np.arange(N), [train_size, valid_size]
    )
    train_dataset = utils.TensorDataset(X[train_index], y[train_index])
    valid_dataset = utils.TensorDataset(X[valid_index], y[valid_index])
    dataloader = utils.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )
    validloader = utils.DataLoader(
        valid_dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )
    train_accs = []
    valid_accs = []
    train_losses = []
    valid_losses = []
    epoch_list = [0]
    best_vloss = float("inf")
#     k = 0
    for i in range(rep):
        print("Training with batch_size: {}, learning_rate: {}".format(batch_size, lr))
        optimizer = optim.SGD(net.parameters(), lr=lr)
        j = 0
        epoch = epoch_list[-1]
        if i >= 1:
            net.load_state_dict(best_state)
        while j < p:
            epoch += 1
            print("epoch: {}".format(epoch))
            net.train_mode()
            for batch in dataloader:
                optimizer.zero_grad()
                X, y = batch
                X = X.view(-1, 3, 64, 64)
                y = y.view(-1)
                X = X.cuda()
                y = y.cuda()

                loss = nn.CrossEntropyLoss()(net.forward(X), y)
                loss.backward()
                optimizer.step()

            net.eval_mode()
            train_loss, train_acc = net.evaluate(dataloader)
            valid_loss, valid_acc = net.evaluate(validloader)

            train_accs.append(train_acc)
            train_losses.append(train_loss)
            valid_losses.append(valid_loss)
            valid_accs.append(valid_acc)
            if valid_loss < best_vloss:
                print("Best epoch yet")
                best_vacc = valid_acc
                best_vloss = valid_loss
                best_state = net.state_dict()
                best_epoch = epoch
                j = 0
            else:
                j += 1

            print(" [LOSS] TRAIN {} / VALID {}".format(train_loss, valid_loss))
            print(" [ACC] TRAIN {} / VALID {}".format(train_acc, valid_acc))
        epoch_list.append(best_epoch)
        # batch_size = int(batch_size / 2)
#         if k % 2 == 0:
#             lr = lr / 2.0
#         else:
#             lr = lr / 5.0
#         k += 1
    best_model = {
        "best_state": best_state,
        "epoch_list": epoch_list,
        "real_epochs": [e + i * p for i, e in enumerate(epoch_list)],
        "batch_size": batch_size,
        "lr": lr,
        "vacc": best_vacc,
        "best_vloss": best_vloss,
        "vloss": valid_losses,
        "taccs": train_accs,
        "tloss": train_losses,
        "vaccs": valid_accs,
        "convs": net.conv_size,
        "lins": net.lin_size,
    }
    return best_model


def accuracy(y_pred, target):
    correct = torch.eq(y_pred.max(1)[1], target).sum().type(torch.FloatTensor)
    return correct / len(target)


class Flatten(nn.Module):
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x


class Dropout(nn.Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        self.p = p

    def forward(self, x):
        data = x.data
        shape = data.shape
        size = int(shape[0] * shape[1])
        drop_idx = np.random.choice(
            np.arange(size), replace=False, size=int(size * self.p)
        )
        data = data.flatten()
        data[drop_idx] = 0
        data = data.reshape(shape)
        x.data = data
        return x


class Net(nn.Module):
    def __init__(self, input_size, conv_size, lin_size):
        super(Net, self).__init__()

        self.drop = True
        self.lin_size = lin_size
        self.conv_size = conv_size
        self.model = nn.Sequential(
            nn.Conv2d(input_size, conv_size, 3, 1),
            nn.ReLU(True),
            nn.Conv2d(conv_size, conv_size, 3, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(conv_size, conv_size * 2, 3, 1),
            nn.ReLU(True),
            nn.Conv2d(conv_size * 2, conv_size * 2, 3, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(conv_size * 2, conv_size * 4, 3, 1),
            nn.ReLU(True),
            nn.Conv2d(conv_size * 4, conv_size * 4, 3, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            Flatten(),
            nn.Linear(conv_size * 4 * 4 * 4, lin_size),
            nn.ReLU(True),
            nn.Linear(lin_size, int(lin_size/2),
            nn.ReLU(True)),
            nn.Linear(int(lin_size/2), 2)
        ).cuda()
#         self.dropout = nn.Sequential(Dropout()).cuda()
#         self.lin1 = nn.Sequential(nn.Linear(lin_size, int(lin_size/2), nn.ReLU(True))).cuda()
#         self.lin2 = nn.Sequential( nn.Linear(int(lin_size/2), 2)).cuda()

    def eval_mode(self):
        self.drop = False

    def train_mode(self):
        self.drop = True

    def forward(self, x):
        return self.model(x)
#         dat = self.model(x)
#         if self.drop:
#             dat = self.dropout(dat)
#         dat = self.lin1(dat)
#         if self.drop:
#             dat = self.dropout(dat)
#         return self.lin2(dat)

    def evaluate(self, dataloader):
        self.eval_mode()
        LOSSES = 0
        ACCURACY = 0
        COUNTER = 0
        for batch in dataloader:
            X, y = batch
            X = X.view(-1, 3, 64, 64)
            y = y.view(-1)
            X = X.cuda()
            y = y.cuda()

            loss = nn.CrossEntropyLoss()(self.forward(X), y)
            acc = accuracy(self.forward(X), y)
            n = y.size(0)
            LOSSES += loss.sum().data.cpu().numpy() * n
            ACCURACY += acc.sum().data.cpu().numpy() * n
            COUNTER += n

        floss = LOSSES / float(COUNTER)
        faccuracy = ACCURACY / float(COUNTER)
        return floss, faccuracy

    def train(self, X, y, epochs, batch_size, lr):
        X = torch.Tensor(X).float()
        y = torch.Tensor(y).long()
        train_dataset = utils.TensorDataset(X, y)
        dataloader = utils.DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True, num_workers=2
        )
        train_accs = []
        train_losses = []
        for i in range(epochs[1:]):
            optimizer = optim.SGD(self.parameters(), lr=lr)
            k=0
            for e in range(epochs[i - 1], epochs[i]):
                self.train_mode()
                for batch in dataloader:
                    optimizer.zero_grad()
                    X, y = batch
                    X = X.view(-1, 3, 64, 64)
                    y = y.view(-1)
                    X = X.cuda()
                    y = y.cuda()

                    loss = nn.CrossEntropyLoss()(self.forward(X), y)
                    loss.backward()
                    optimizer.step()

                self.eval_mode()
                train_loss, train_acc = self.evaluate(dataloader)

                train_accs.append(train_acc)
                train_losses.append(train_loss)

                print("epoch: {}".format(e))
                print("TRAIN : [LOSS] {} / [ACC] {}".format(train_loss, train_acc))
#             batch_size = int(batch_size / 2)
#             if k % 2 == 0:
#                 lr = lr / 5.0
#             else:
#                 lr = lr / 2.0
#             k += 1
        print("Training Done")



def create_dataset(cats_path, dogs_path, test_path):
    train_set = []
    target = []
    n_cats = len(os.listdir(cats_path))
    for i in range(1, n_cats):
        f = '{}.Cat.jpg'.format(i)
        mat = img.imread(cats_path + f)
        if not len(mat.shape) == 3:
            mat = np.array((mat, mat, mat)).T
        train_set.append(mat)
        target.append(0)

    n_dogs = len(os.listdir(dogs_path))
    for i in range(1, n_dogs):
        f = '{}.Dog.jpg'.format(i)
        mat = img.imread(dogs_path + f)
        if not len(mat.shape) == 3:
            mat = np.array((mat, mat, mat)).T
        train_set.append(mat)
        target.append(1)

    train_set = np.asarray(train_set)
    target = np.asarray(target)
    
    test_set = []
    n_test = len(os.listdir(test_path))
    for i in range(1, n_test):
        f = '{}.jpg'.format(i)
        mat = img.imread(test_path + f)
        if not len(mat.shape) == 3:
            mat = np.array((mat, mat, mat)).T
        test_set.append(mat)

    return train_set/255.0, target, np.asarray(test_set)/255.0

In [3]:
test_path = "/home/arthur/git/IFT6135_assignment_1/testset/test/"
dogs_path = "/home/arthur/git/IFT6135_assignment_1/trainset/Dog/"
cats_path = "/home/arthur/git/IFT6135_assignment_1/trainset/Cat/"
X, y, submission = create_dataset(cats_path, dogs_path, test_path)
idx = np.random.permutation(len(X))
X = X[idx]
y = y[idx]

In [4]:
net = Net(3, 64, 512)
par = sum(p.numel() for p in net.parameters() if p.requires_grad)

print("The network contains {} parameters.".format(par))

The network contains 3374914 parameters.


In [5]:
batch_size = 70
learning_rate = 0.001

result = train(net, X, y, batch_size, learning_rate, p=15, rep=1)

Training with batch_size: 70, learning_rate: 0.001
epoch: 1
Best epoch yet
 [LOSS] TRAIN 0.6932245090448159 / VALID 0.6930623888969422
 [ACC] TRAIN 0.4992776245945267 / VALID 0.506500009894371
epoch: 2
 [LOSS] TRAIN 0.6931966474968265 / VALID 0.6930687415599823
 [ACC] TRAIN 0.49927762343528986 / VALID 0.5065000066161156
epoch: 3
 [LOSS] TRAIN 0.6931783003617881 / VALID 0.6930777424573898
 [ACC] TRAIN 0.49927762424675565 / VALID 0.5065000051259995
epoch: 4


KeyboardInterrupt: 

In [ ]:
plt.plot(range(len(result["tloss"])), result["tloss"], label='training loss')
plt.plot(range(len(result["tloss"])), result["vloss"], label='validation loss')
for e in result['real_epochs']:
    plt.axvline(x=e, label='learning rate change' if e==0 else None, color='green')
plt.axvline(x=result['real_epochs'][-2], label='early stop', color='black')
plt.legend()
plt.show()

In [ ]:
plt.plot(range(len(result["taccs"])), result["taccs"], label='training accuracy')
plt.plot(range(len(result["vaccs"])), result["vaccs"], label='validation accuracy')
for e in result['real_epochs']:
    plt.axvline(x=e, label='learning rate change' if e==0 else None, color='green')
plt.axvline(x=result['real_epochs'][-2], label='early stop', color='black')
plt.legend()
plt.show()

In [ ]:
from testlab import labs

labs = torch.Tensor(labs)
batch = torch.Tensor(submission[:500])
batch = batch.view(-1, 3, 64, 64)
batch = batch.cuda()

print(accuracy(net.forward(batch), labs.long().cuda()))

In [ ]:
test_net = Net(3, 32, 512)
test_net.train(X, y, result['epoch_list'], batch_size, learning_rate)
split = 10
step = int(len(submission) / split)
for i in range(0, len(submission), step):
    batch = torch.Tensor(submission[i : i + step])
    batch = batch.view(-1, 3, 64, 64)
    batch = batch.cuda()

    y += list(map(int, test_net.forward(batch).max(1)[1].cpu()))

y = pd.DataFrame(y)
y.index.name = "id"
y.columns = ["label"]
y = y.replace(0, "Cat")
y = y.replace(1, "Dog")
y.index += 1
y.to_csv("submission.csv")

In [ ]:
test_net = Net(3, 32, 512)
test_net.load_state_dict(result['best_state'])
split = 10
step = int(len(submission) / split)
y = []
for i in range(0, len(submission), step):
    batch = torch.Tensor(submission[i : i + step])
    batch = batch.view(-1, 3, 64, 64)
    batch = batch.cuda()

    y += list(map(int, test_net.forward(batch).max(1)[1].cpu()))

y = pd.DataFrame(y)
y.index.name = "id"
y.columns = ["label"]
y = y.replace(0, "Cat")
y = y.replace(1, "Dog")
y.index += 1
y.to_csv("submission.csv")

In [ ]:
result.keys()